In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import requests
import csv

In [9]:
def read_openbci_txt_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    data_lines = lines[4:]  # Omitir las primeras líneas de encabezado
    data = []
    for line in data_lines:
        parts = line.strip().split(',')
        # Convertir todas las columnas excepto las dos últimas (timestamp y hora)
        float_parts = list(map(float, parts[:-2]))
        data.append(float_parts)
    return np.array(data)

txt_file_path = 'OpenBCI_GUI-v5-ojos abiertos_cerrados.txt'
data = read_openbci_txt_file(txt_file_path)

# Validar la lectura de datos
print(data.shape)
print(data[:5])  # Mostrar las primeras 5 filas para verificar


(22491, 22)
[[ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.137936e+04  4.949289e+04 -1.659706e+04 -2.130975e+04
   6.703910e+03 -3.284860e+03  7.223100e+03  1.740110e+03  4.000000e-02
   4.200000e-01  2.380000e-01  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.097346e+04  4.897247e+04 -1.627902e+04 -2.105013e+04
   7.045020e+03 -2.887530e+03  7.593090e+03  2.118700e+03  4.000000e-02
   4.200000e-01  2.380000e-01  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  0.000000e+00]
 [ 2.000000e+00  6.143345e+04  

In [10]:
def find_most_varying_signal(data):
    # Ignorar la primera columna (número de muestra) y las últimas dos columnas (timestamp y hora)
    signal_data = data[:, 1:-2]
    variances = np.var(signal_data, axis=0)
    most_varying_signal_index = np.argmax(variances) + 1  # +1 porque ignoramos la primera columna
    return most_varying_signal_index

most_varying_signal_index = find_most_varying_signal(data)
print(f'La señal que más varía es la columna {most_varying_signal_index}')

La señal que más varía es la columna 5


In [11]:
def create_csv_windows(data, signal_index, window_size, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    num_windows = len(data) // window_size
    for i in range(num_windows):
        window_data = data[i * window_size:(i + 1) * window_size, signal_index]
        output_file = os.path.join(output_folder, f'EMG.sig_{i}.csv')
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['timestamp', 'signal'])
            for j, value in enumerate(window_data):
                writer.writerow([j, value])

# Definir el tamaño de la ventana y la carpeta de salida
Fs = 1000
window_size = 2*Fs  # Como hay mucha duración, tomaré como ventaneo 2 segundos
output_folder = 'output_windows_openbci_ejercicios'
create_csv_windows(data, most_varying_signal_index, window_size, output_folder)
print(f'Se han creado los archivos CSV en la carpeta {output_folder}')

Se han creado los archivos CSV en la carpeta output_windows_openbci_ejercicios


In [12]:
def upload_files_to_edge_impulse(api_key, label, folder_path):
    url = 'https://ingestion.edgeimpulse.com/api/training/files'
    headers = {
        'x-api-key': api_key,
        'x-label': label,
    }
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    for file_path in files:
        with open(file_path, 'rb') as file:
            files_payload = {'data': (os.path.basename(file_path), file, 'application/csv')}
            res = requests.post(url, headers=headers, files=files_payload)
            if res.status_code == 200:
                print(f'Uploaded {file_path} to Edge Impulse successfully.')
            else:
                print(f'Failed to upload {file_path} to Edge Impulse. Status code: {res.status_code}')

api_key = 'ei_51b8790275170673d369726381038ceadcc54546810ff47d8eb0e78978783422'
label = 'eeg_ejericios_ULtracortex'
upload_files_to_edge_impulse(api_key, label, output_folder)

Uploaded output_windows_openbci_ejercicios\EMG.sig_0.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_1.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_10.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_2.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_3.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_4.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_5.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_6.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_7.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_8.csv to Edge Impulse successfully.
Uploaded output_windows_openbci_ejercicios\EMG.sig_9.csv to Edge Impulse successfully.
